# РК № 2 Сафиуллин А.П. 
# РТ5-61Б Вариант № 14
# Задание.
Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.
# Метод №1
Дерево решений
# Метод №2
Градиентный бустинг


# О наборе данных:
Этот набор данных предназначен для объединения нескольких аспектов данных об образовании в США в один удобный файл CSV (states_all.csv).
* GRADES_PK: количество учащихся дошкольного образования.

* GRADES_4: количество учащихся четвертого класса.

* GRADES_8: количество учащихся восьмого класса.

* GRADES_12: количество учащихся двенадцатого класса.

* GRADES_1_8: количество учащихся с первого по восьмой классы.

* GRADES_9_12: Количество учащихся с девятого по двенадцатый классы.

* GRADES_ALL: количество всех учащихся в штате. Сопоставимо с ENROLL в финансовых данных (это США. оценка Бюро переписи населения для студентов в штате).
* AVG_MATH_4_SCORE: средний балл по штату для четвероклассников, сдающих экзамен по математике NAEP.

* AVG_MATH_8_SCORE: средний балл восьмиклассников штата, сдающих экзамен по математике NAEP.

* AVG_READING_4_SCORE: средний балл по штату для четвероклассников, сдающих экзамен по чтению NAEP.

* AVG_READING_8_SCORE: средний балл по штату для восьмиклассников, сдающих экзамен по чтению NAEP.

In [369]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))



Подключаем и смотрим характеристики датасета

In [370]:
data = pd.read_csv("/kaggle/input/us-education-datasets-unification-project/states_all.csv")
data.info()

Размер набора данных

In [371]:
data.shape

Количество пустых значений в датасете

In [372]:
data.isnull().sum()

In [373]:
data.head()

In [374]:
data.describe()

**Т.к. все колоки с пустыми значениями являются числовыми, заполним пропуски нулями**

In [375]:
data_new_1 = data.fillna(0)
data_new_1.head()

Как видно далее пропуски были заполнены

In [376]:
data_new_1.isnull().sum()

Функция describe оценивает столбцы с числовым типом данных (int или float) и возвращает статистические данные, которые дают представление о распределении значений.

In [377]:
data_new_1.describe().round(2)

Выведем количество уникальных значений по колонке STATE

In [378]:
data_new_1["STATE"].value_counts()

In [379]:
data_new_1["STATE"].value_counts(normalize=True)

# Построим Корреляционную матрицу

In [380]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

Критерий корреляции Пирсона – это метод параметрической статистики, позволяющий определить наличие или отсутствие линейной связи между двумя количественными показателями, а также оценить ее тесноту и статистическую значимость. Другими словами, критерий корреляции Пирсона позволяет определить, изменяется ли (возрастает или уменьшается) один показатель в ответ на изменения другого.
Используем Пирсона т.к. он является наиболее универсальным

In [381]:

ig, ax = plt.subplots(figsize=(30,25))
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html
sns.heatmap(data_new_1.corr(method='pearson'), ax=ax, annot=True, fmt='.3f', cbar_kws= {'orientation': 'horizontal'})

# Значения коэффициента корреляции интерпретируются следующим образом:

* До 0,2 — очень слабая корреляция
* До 0,5 — слабая
* До 0,7 — средняя
* До 0,9 — высокая
* Больше 0,9 — очень высокая
# Корреляция:
Можно наблюдать слабую корреляцию последних 4 колонок от всех остальных. т.к. последние 4 колонки показывают средую качественную оценку экзамена, тогда как остальные выражают количественную оценку учеников 

In [382]:
data_new_1_1 = data_new_1.drop(columns=['YEAR','AVG_READING_8_SCORE','AVG_READING_4_SCORE','AVG_MATH_8_SCORE','AVG_MATH_4_SCORE'])
data_new_1_2 = data_new_1.drop(columns=['ENROLL', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G'])

In [383]:
ig, ax = plt.subplots(figsize=(30,25))
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html
sns.heatmap(data_new_1_1.corr(method='pearson'), ax=ax, annot=True, fmt='.3f', cbar_kws= {'orientation': 'horizontal'}, vmin = 0)

In [384]:
ig, ax = plt.subplots(figsize=(10,10))
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html
sns.heatmap(data_new_1_2.corr(method='pearson'), ax=ax, annot=True, fmt='.3f', vmin = 0)

In [385]:
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn import metrics 

In [386]:
data_new_1['STATE'].value_counts()

In [387]:
data_new_1.columns

In [388]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets, preprocessing
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,  balanced_accuracy_score, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

Будем решать задачу регрессии

In [389]:
with sns.axes_style('whitegrid'):
 sns.countplot(x='STATE', data=data_new_1, palette='Accent')
 plt.show()

In [390]:
from sklearn.preprocessing import LabelEncoder

In [391]:
data_new_1["STATE"].unique()
le = LabelEncoder()
state_enc_le = le.fit_transform(data_new_1["STATE"])

In [392]:
le.classes_

In [393]:
np.unique(state_enc_le)

In [394]:

data_new_1_3 = data_new_1.drop(columns = ['PRIMARY_KEY','STATE'])
label = state_enc_le
X_train, X_test, y_train, y_test = train_test_split(
    data_new_1_3, label, test_size=0.3, random_state=1)

In [395]:
y_train

# Метрики качества классификации

In [396]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor, export_graphviz, export_text
from sklearn.model_selection import train_test_split, GridSearchCV

# Метод № 1

# Дерево решений


Решающие деревья — ещё один алгоритм машинного обучения. Обучение алгоритма представляет собой пошаговое разбиение обучающей выборки на части (чаще всего на две, но в общем случае это не обязательно) по какому-либо признаку. 

# Композиция решающих деревьев
* Решающие деревья могут выявить очень сложные закономерности в обучающей выборке. Но чем лучший результат обучающие деревья показывают на обучении, тем худший результат они покажут на новых данных — деревья переобучаются. Для устранения этой проблемы придуман механизм объединения обучающих деревьев в композиции (леса).

* Для построения композиции деревьев берется N обученных деревьев и их результат “усредняется”. Для задач регрессии (там, где предсказываем число) берется среднее значение предсказанных чисел, а для задач классификации — самый популярный (из N предсказаний) предсказанный класс.
* Таким образом, для построения леса необходимо сначала обучить N решающих деревьев. Но делать это на одной и той же обучающей выборке бессмысленно: мы получим N одинаковых алгоритмов и результат усреднения будет равен результату любого из них. Поэтому для обучения базовых деревьев используется рандомизация: обучение каждого базового дерева производится по случайной подвыборке объектов исходной обучающей выборки и/или по случайному набору параметров (т.е. для обучения каждого базового алгоритма берутся не все параметры объектов, а только случайный набор определенного размера). Но и этого бывает недостаточно для построения независимых алгоритмов — применяют рандомизацию признаков для разбиения на каждой вершине (т.е. на каждой вершине каждого дерева признак выбирают не из всего набора, а из случайной подвыборки признаков определенного размера; важно, что для каждой вершины каждого дерева случайная подвыборка — своя). Поэтому алгоритм и называется композицией случайных деревьев.

# GridSearchCV (решетчатый поиск) param:
* estimator: Предполагается, что это реализует интерфейс оценщика scikit-learn. Либо оценщик должен предоставить функцию оценки, либо оценка должна быть передана. В нашом случае оценщик - дерево
* cv: количество частей, на которые разбиваем выборку для кросс-валидации; их ещё называют "folds"
* param_griddict: 


In [397]:
params = {'min_samples_leaf': range(3, 30)}
tree = DecisionTreeRegressor(random_state=3)
grid_cv = GridSearchCV(estimator=tree, cv=5, param_grid=params, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_cv.fit(X_train, y_train)
print(grid_cv.best_params_)

In [457]:
def print_metrics(y_test, y_pred):
    print(f"R^2: {r2_score(y_test, y_pred)}")
    print(f"RMSE: {mean_squared_error(y_test, y_pred)}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

In [459]:
best_tree = grid_cv.best_estimator_
best_tree.fit(X_train, y_train)
y_pred_tree = best_tree.predict(X_test)
y_pred_tree_1 = best_tree.predict(X_train)
print_metrics(y_test, y_pred_tree)

# Метод № 2

# Градиентный бустинг

In [444]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [445]:
abc =  AdaBoostClassifier()
abc.fit(X_train,y_train)

In [458]:
y_pred_abc = abc.predict(X_test)
y_pred_abc_1 = abc.predict(X_train)
print_metrics(y_test, y_pred_abc)

In [446]:
accuracy_score(y_train,y_pred_abc_1)

In [447]:
accuracy_score(y_test,y_pred_abc)